In [ ]:
# default_exp core

In [ ]:
#hide
from nbdev.showdoc import *

# ProofZero Core SDK

A feature parser-hasher client to the Proof Zero matching engine.

## Benefits

Generating matches across datasets is difficult. The same entities are described by different data (for example, one person using different phone numbers, or addresses). Data are often in different formats -- sometimes just a postal code is stored, sometimes a geo-region, sometimes a street address.

However all of these differences in physical data representation refer to the same logical entities.

The Proof Zero SDK generates pseudokeys that match entities across data sets containing different physical data representations. These pseudokeys enable a match between incomplete, incorrect, and unclean data sets. The percent strength of these pseudokey-based matches is also indicated.

## Workflow

In order to join two datasets using pseudokeys we first `load` our data, and then:

1. `tokenize` it -- decompose our data into standard formats,
2. `index` it -- recompose our tokens into cryptographic indexes,
3. `match` it -- compare indexed data sets to generate matches and measure match quality.

This is the "TIM" workflow: `tokenize`, `index`, `match`.

## Security and data privacy

All SDK features run locally, so data remain within control of the user. match optionally allows use of Proof Zero's compute cluster and matching models to generate matches more quickly and with higher accuracy.

By using IndexFrame objects data remain in the control of the local system, even when using Proof Zero's compute cluster.

## Exported types

We provide several lightweight types that track progress through the TIM workflow:

* `TokenFrame` is a `pandas` DataFrame that has been tokenized.
* `IndexFrame` is a `TokenFrame` that has been hashed/indexed.
* `MatchFrame` is a match between two `IndexFrame`s.

These types help systems and analysts keep track of where a given dataset is in the tokenize, index, match (TIM) workflow.

## Loading data

We load our data into standard `pandas` DataFrames.

In [ ]:
#export
import pandas as pd
def load(filename: str) -> pd.DataFrame:
    """
    Convenience function for loading files using `pandas`. Full version supports XLSX, etc.
    """
    return pd.read_csv(filename)

In [ ]:
data_1 = load('./data_1.csv')
assert (data_1[:1].equals(pd.DataFrame({'acct_num': 'PXCG66212484637575', 'name': 'Kristin Sanchez MD', 'address': '31417 Gina Lodge, Bradleytown, MB P7G 4N5', 'phone': '1 (598) 742-6794', 'SIN': '203 268 552', 'DOB': '1943/04/15'}, index=[0])))
data_1[:1]

,acct_num,name,address,phone,SIN,DOB
0,PXCG66212484637575,Kristin Sanchez MD,"31417 Gina Lodge, Bradleytown, MB P7G 4N5",1 (598) 742-6794,203 268 552,1943/04/15


## Tokenizing data

We apply parsers to the DataFrames we want to process in order to decompose the physical data into standard types. This lets us re-compose a hashed index later that encodes knowledge of the underlying data structure.

In [ ]:
#export
import itertools
import functools
from typing import Callable, NewType

# The `TokenFrame` type is a wrapper around a `pandas` DataFrame that represents a DataFrame that has been tokenized using the `tokenize` function.
TokenFrame = NewType('TokenFrame', pd.DataFrame)

def tokenize(
    df: pd.DataFrame, schema: dict, suffix_delim: str = ""
) -> TokenFrame:
    """
    Takes a `pandas` DataFrame and a schema. The schema is a `dict` that maps columns in the DataFrame to a list of parsers that are executed in order.
    
    Denormalizes the passed DataFrame by applying the parsers in the schema.
    
    Returns the denormalized DataFrame.
    """
    def map_schema(map_row, map_schema, map_delim):
        results = [
            functools.reduce(
                lambda data, fxn: fxn(data), map_schema[i], map_row[i]
            )
            for i in map_row.index
        ]
        indicies = [
            # Allow parsers to return a named component and use that name to index
            # our new columns, else use the index number as a string.
            map_delim.join(
                [map_row.index[i], v[0] if isinstance(v, tuple) else str(j)]
            )
            for i, u in enumerate(map_row.index)
            for j, v in enumerate(results[i])
        ]
        series = pd.Series(
            data=list(itertools.chain(*results)), index=indicies
        )
        return series

    return pd.concat(
        [
            df,
            df.apply(
                map_schema,
                axis=1,
                args=(schema, suffix_delim),
                result_type="expand",
            ),
        ],
        axis=1,
    )

In [ ]:
from proofzero_sdk.util import parseString, parseName, parseAddress, parsePhone, parseSIN, parseDate
tokens_1 = tokenize(df=data_1[:1], schema={
    'acct_num': [parseString],
    'name': [parseName],
    'address': [parseAddress],
    'phone': [parsePhone],
    'SIN': [parseSIN],
    'DOB': [parseDate]
}, suffix_delim='_')
tokens_1[:1]

,acct_num,name,address,phone,SIN,DOB,acct_num_0,name_title,name_first,name_middle,...,phone_10,phone_None,SIN_203-268-552,SIN_203,SIN_268,SIN_552,DOB_1943-04-15,DOB_1943,DOB_4,DOB_15
0,PXCG66212484637575,Kristin Sanchez MD,"31417 Gina Lodge, Bradleytown, MB P7G 4N5",1 (598) 742-6794,203 268 552,1943/04/15,PXCG66212484637575,"(title, )","(first, Kristin)","(middle, )",...,"(10, 10)","(None, 11)","(203-268-552, 0)","(203, 1)","(268, 2)","(552, 3)","(1943-04-15, 0)","(1943, 1)","(4, 2)","(15, 3)"


## Indexing data

In [ ]:
#export

from proofzero_sdk.util import sha2

# The `IndexFrame` type is a wrapper around a `TokenFrame` (ie, a `pandas` DataFrame) that represents a tokenized DataFrame that has been indexed using the `index` function.
IndexFrame = NewType('IndexFrame', TokenFrame)

def index(
    df: TokenFrame, schema: dict, hasher: Callable = sha2, suffix_delim: str = ""
) -> IndexFrame:
    """
    Tokenizes the passed DataFrame using `tokenize`, then applies a hash function to the tokenized features.
    
    The default hash function is `sha2`, from the Proof Zero utility SDK.
    """
    return tokenize(df, schema, suffix_delim).applymap(hasher)

In [ ]:
index_1 = index(df=data_1[:1], schema={
    'acct_num': [parseString],
    'name': [parseName],
    'address': [parseAddress],
    'phone': [parsePhone],
    'SIN': [parseSIN],
    'DOB': [parseDate]
}, suffix_delim='_')
index_1[:1]

,acct_num,name,address,phone,SIN,DOB,acct_num_0,name_title,name_first,name_middle,...,phone_10,phone_None,SIN_203-268-552,SIN_203,SIN_268,SIN_552,DOB_1943-04-15,DOB_1943,DOB_4,DOB_15
0,9a4e0b4212e6e1c3bee9d9477ada501d4af3278253b92c...,e4fb3b4959e9a84de7c2ca8979550028303df30d40a2a6...,f1786bd10278eb29886cc1dfc7d1525ca1b46f32837cb9...,5776f839fd13f8177c3e770a636d1912f5388b0df0d5de...,4dc80e9334c9dd570992acb20cc61f223d3b74b1b2e985...,cac1d345f8d91e8b2ef0daf185e59d5a999d067a526317...,9a4e0b4212e6e1c3bee9d9477ada501d4af3278253b92c...,bc1f4845d8154f8f8cfcdc9b2b5d87a338ea4a938eb33e...,127a5d72c7b76ea92ef1b812842ca65c427b294dde3401...,40fc1952b9a79fc35e8fcea702c00f764eaf2ec0fce420...,...,7ae4b082a66b4fbadeb161e0d671e0ec8d40eae0dc71c8...,e92705293fc34d3a9c7bba3f04caea339edc52f7b899b2...,cf4ca7b336ad79aef924e8ecdfbd857cd40b8449f4183c...,51c6b7349d874760a41fb598de77f706d494fbf890b7d0...,80c4546bc9831ac81bb0d78fcbd93824dd5510339eae64...,ed0706dd57a13c0b65aee46fb399ae12909f9fcd23e081...,69fe02c8a53e77d0ee77b602ba1afdb3f5a0f1f96d64a8...,79026fe2dfa0c65ed203a34f024edb075a0dbce04fa7cf...,2a69ad49abd3193c629fa56c6647e5fe9718e7b0051f07...,c861c3b4dcd6234a9199867826bd041d449a4f355e0f14...


## Matching data

In [ ]:
#export

# The `MatchFrame` type is a wrapper around a `IndexFrame` (ie, a `TokenFrame` and ultimately a `pandas` DataFrame) that represents an indexed DataFrame that has been matched using the `match` function.
MatchFrame = NewType('MatchFrame', IndexFrame)

def match(
    index_0: IndexFrame, index_1: IndexFrame, network: bool = False
) -> pd.DataFrame:
    """
    Discover matches between the two passed DataFrames. If `network` is set to `True` the indexed data are matched using Proof Zero's cluster.
    """
    return pd.DataFrame()

In [ ]:
data_2 = load('./data_2.csv')
data_2 = data_2.iloc[[2]]
data_2

,acct_num,name,address,phone,SIN,DOB
2,CUVC30129179791293,Kristin Sanchez MD,"31417 Gina Lodge, Bradleytown, MB P7G 4N5",742 598 6794,203 268 552,15811


In [ ]:
tokens_2 = tokenize(df=data_2[:1], schema={
    'acct_num': [parseString],
    'name': [parseName],
    'address': [parseAddress],
    'phone': [parsePhone],
    'SIN': [parseSIN],
    'DOB': [parseDate]
}, suffix_delim='_')
tokens_2

TypeError: Input type must be str

In [ ]:
index_2 = index(df=data_2[:1], schema={
    'acct_num': [parseString],
    'name': [parseName],
    'address': [parseAddress],
    'phone': [parsePhone],
    'SIN': [parseSIN],
    'DOB': [parseDate]
}, suffix_delim='_')
index_2

TypeError: Input type must be str

In [ ]:
match_frame = match(index_1, index_2)
match_frame